# Overview

In [13]:
using LightGraphs, TraceCalls
A = [
    0 1 1
    1 0 1
    1 1 0
]
g = Graph(A)
@trace LightGraphs adjacency_matrix(g)

TraceCalls.Trace(TraceCalls.#18, (), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.adjacency_matrix, ({3, 3} undirected simple Int64 graph, :out, Int64), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.indegree, ({3, 3} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], 2), TraceCalls.Trace(LightGraphs.indegree, ({3, 3} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], 2), TraceCalls.Trace(LightGraphs.indegree, ({3, 3} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], 2), TraceCalls.Trace(LightGraphs.indegree, ({3, 3} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], 2), TraceCalls.Trace(LightGraphs.indegree, ({3, 3} undirected simple Int64 graph, 3), (), TraceCalls.Trace[], 2), TraceCalls.Trace(LightGraphs.indegree, ({3, 3} undirected simple Int64 graph, 3), (), TraceCalls.Trace[], 2)], 
  [2, 1]  =  1
  [3, 1]  =  1
  [1, 2]  =  1
  [3, 2]  =  1
  [1, 3]  =  1
  [2, 3]  =  1)], 
  [2, 1]  =  1
  [3, 1]  =  1
  [1, 2]  =  1
  [3, 2]  =  1
  [1, 3]  =  1
  [2, 3]  =  1)

In [12]:
@trace LightGraphs randomwalk(g, 1, 10)

TraceCalls.Trace(TraceCalls.#18, (), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.randomwalk, ({3, 3} undirected simple Int64 graph, 1, 10), (), TraceCalls.Trace[], [1, 2, 1, 3, 1, 2, 3, 2, 1, 2])], [1, 2, 1, 3, 1, 2, 3, 2, 1, 2])

In [7]:
@which add_edge!(g, 1, 2)

add_edge!(g::LightGraphs.SimpleGraphs.AbstractSimpleGraph, x, y) in LightGraphs.SimpleGraphs at /Users/cedric/.julia/v0.6/LightGraphs/src/graphtypes/simplegraphs/SimpleGraphs.jl:62

In [2]:
A = [
    0 1 1
    1 0 1
    1 1 0
]

@trace LightGraphs Graph(A)

TraceCalls.Trace(TraceCalls.#18, (), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs._insert_and_dedup!, ([2, 3], 2), (), TraceCalls.Trace[], true), TraceCalls.Trace(LightGraphs._insert_and_dedup!, ([1, 3], 1), (), TraceCalls.Trace[], true), TraceCalls.Trace(LightGraphs._insert_and_dedup!, ([2, 3], 3), (), TraceCalls.Trace[], true), TraceCalls.Trace(LightGraphs._insert_and_dedup!, ([1, 2], 1), (), TraceCalls.Trace[], true), TraceCalls.Trace(LightGraphs._insert_and_dedup!, ([1, 3], 3), (), TraceCalls.Trace[], true), TraceCalls.Trace(LightGraphs._insert_and_dedup!, ([1, 2], 2), (), TraceCalls.Trace[], true)], {3, 3} undirected simple Int64 graph)

In [3]:
upd = TraceCalls.traceable_update(LightGraphs);

In [3]:
Pkg.test("ClobberingReload")

INFO: Testing ClobberingReload
INFO: Reloading AA
INFO: Reloading BB
INFO: Reloading ParametricTypeAlias
INFO: ClobberingReload tests passed


In [1]:
using TraceCalls

In [2]:
using LightGraphs
A = [
    0 1 1
    1 0 1
    1 1 0
]
graph = Graph(A)
TraceCalls.tree_size(@trace LightGraphs adjacency_matrix(graph)) > 5


true

In [6]:
tr = @trace LightGraphs is_connected(graph);

In [28]:
@which LightGraphs.DummyEdgeMap()

LightGraphs.DummyEdgeMap() in LightGraphs at /Users/cedric/.julia/v0.6/LightGraphs/src/traversals/graphvisit.jl:35

In [27]:
tr[1,2,2].args[1]

MethodError: [91mMethodError: no method matching length(::LightGraphs.DummyEdgeMap)[0m
Closest candidates are:
  length([91m::SimpleVector[39m) at essentials.jl:256
  length([91m::Base.MethodList[39m) at reflection.jl:558
  length([91m::MethodTable[39m) at reflection.jl:634
  ...[39m

In [14]:
Pkg.test("TraceCalls")

INFO: Computing test dependencies for TraceCalls...
INFO: No packages to install, update or remove
INFO: Testing TraceCalls


Max 5 x + 3 y
Subject to
 x + 5 y ≤ 3
 0 ≤ x ≤ 2
 0 ≤ y ≤ 30
Objective value: 10.6
x = 2.0
y = 0.2


ERROR: LoadError: invalid redefinition of constant g
Stacktrace:
 [1] include_from_node1(::String) at ./loading.jl:569
 [2] include(::String) at ./sysimg.jl:14
 [3] process_options(::Base.JLOptions) at ./client.jl:305
 [4] _start() at ./client.jl:371
while loading /Users/cedric/.julia/v0.6/TraceCalls/test/runtests.jl, in expression starting on line 98
=============================[ ERROR: TraceCalls ]==============================

failed process: Process(`/Applications/Julia-0.6.app/Contents/Resources/julia/bin/julia -Ccore2 -J/Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib --compile=yes --depwarn=yes --check-bounds=yes --code-coverage=none --color=yes --compilecache=yes /Users/cedric/.julia/v0.6/TraceCalls/test/runtests.jl`, ProcessExited(1)) [1]

INFO: No packages to install, update or remove


LoadError: [91mTraceCalls had test errors[39m

In [3]:
using TraceCalls, DataStructures

trace_ordered_dict = @trace DataStructures SortedDict(1=>2, 3=>4)

TraceCalls.Trace(TraceCalls.#18, (), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.SortedDict, (1=>2, 3=>4), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.SortedDict, (Base.Order.ForwardOrdering(), (1=>2, 3=>4)), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures._sorted_dict_with_eltype, (Base.Order.ForwardOrdering(), (1=>2, 3=>4), Pair{Int64,Int64}), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.initializeTree!, (DataStructures.TreeNode{Int64}[DataStructures.TreeNode{Int64}(1, 3, 0, 3, 1, 1), DataStructures.TreeNode{Int64}(4, 2, 0, 3, 1, 1), DataStructures.TreeNode{Int64}(1, 2, 0, 0, 3, 3)],), (), TraceCalls.Trace[], nothing), TraceCalls.Trace(DataStructures.initializeData!, (DataStructures.KDRec{Int64,Int64}[DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 1, 2), DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 3, 4)],), (), TraceCalls.Trace[], nothing), TraceCalls.Trace(push!, (IntSet([1, 2, 3, 4]), 1, 2), (), TraceCalls.Trace[TraceCalls.Trace(push!, (IntSet([1, 2, 3, 4]), 1), (), TraceCalls.Trace[], IntSet([1, 2, 3, 4])), TraceCalls.Trace(push!, (IntSet([1, 2, 3, 4]), 2), (), TraceCalls.Trace[], IntSet([1, 2, 3, 4]))], IntSet([1, 2, 3, 4])), TraceCalls.Trace(insert!, (DataStructures.BalancedTree23{Int64,Int64,Base.Order.ForwardOrdering}(Base.Order.ForwardOrdering(), DataStructures.KDRec{Int64,Int64}[DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 1, 2), DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 3, 4)], DataStructures.TreeNode{Int64}[DataStructures.TreeNode{Int64}(1, 3, 0, 3, 1, 1), DataStructures.TreeNode{Int64}(4, 2, 0, 3, 1, 1), DataStructures.TreeNode{Int64}(1, 2, 0, 0, 3, 3)], 3, 2, Int64[], Int64[], IntSet([1, 2, 3, 4]), [4950670392, 4664686416, 4611129552], [4950669912, 4664686416, 4611129552]), 1, 2, false), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.findkey, (DataStructures.BalancedTree23{Int64,Int64,Base.Order.ForwardOrdering}(Base.Order.ForwardOrdering(), DataStructures.KDRec{Int64,Int64}[DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 1, 2), DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 3, 4)], DataStructures.TreeNode{Int64}[DataStructures.TreeNode{Int64}(1, 3, 0, 3, 1, 1), DataStructures.TreeNode{Int64}(4, 2, 0, 3, 1, 1), DataStructures.TreeNode{Int64}(1, 2, 0, 0, 3, 3)], 3, 2, Int64[], Int64[], IntSet([1, 2, 3, 4]), [4950670392, 4664686416, 4611129552], [4950669912, 4664686416, 4611129552]), 1), (), TraceCalls.Trace[], (1, false)), TraceCalls.Trace(DataStructures.push_or_reuse!, (DataStructures.KDRec{Int64,Int64}[DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 1, 2), DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 3, 4)], Int64[], DataStructures.KDRec{Int64,Int64}(0, 1, 2)), (), TraceCalls.Trace[], 3), TraceCalls.Trace(DataStructures.replaceparent!, (DataStructures.KDRec{Int64,Int64}[DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 1, 2), DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 3, 4)], 3, 1), (), TraceCalls.Trace[], nothing), TraceCalls.Trace(push!, (IntSet([1, 2, 3, 4]), 3), (), TraceCalls.Trace[], IntSet([1, 2, 3, 4]))], (true, 3)), TraceCalls.Trace(insert!, (DataStructures.BalancedTree23{Int64,Int64,Base.Order.ForwardOrdering}(Base.Order.ForwardOrdering(), DataStructures.KDRec{Int64,Int64}[DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 1, 2), DataStructures.KDRec{Int64,Int64}(1, 1, 2), DataStructures.KDRec{Int64,Int64}(2, 3, 4)], DataStructures.TreeNode{Int64}[DataStructures.TreeNode{Int64}(1, 3, 0, 3, 1, 1), DataStructures.TreeNode{Int64}(4, 2, 0, 3, 1, 1), DataStructures.TreeNode{Int64}(1, 2, 0, 0, 3, 3)], 3, 2, Int64[], Int64[], IntSet([1, 2, 3, 4]), [4950670392, 4664686416, 4611129552], [4950669912, 

In [2]:
measure(:@elapsed, trace_ordered_dict)

TraceCalls.Trace(TraceCalls.#18, (), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.OrderedDict, (1=>2, 3=>4), (), TraceCalls.Trace[TraceCalls.Trace(setindex!, (DataStructures.OrderedDict(1=>2,3=>4,1=>2,3=>4), 2, 1), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.ht_keyindex2, (DataStructures.OrderedDict(1=>2,3=>4,1=>2,3=>4), 1), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.hashindex, (1, 16), (), TraceCalls.Trace[], TraceCalls.FontColor("4fb000", 0.3094))], TraceCalls.FontColor("ff0000", 1.6643)), TraceCalls.Trace(DataStructures._setindex!, (DataStructures.OrderedDict(1=>2,3=>4,1=>2,3=>4), 2, 1, 16), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.rehash!, (DataStructures.OrderedDict(1=>2,3=>4,1=>2,3=>4), 4), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures._tablesz, (4,), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0.0006)), TraceCalls.Trace(length, (DataStructures.OrderedDict(1=>2,3=>4,1=>2,3=>4),), (), TraceCalls.Trace[], TraceCalls.FontColor("3fc000", 0.2455)), TraceCalls.Trace(DataStructures.hashindex, (1, 16), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0.0003))], TraceCalls.FontColor("00ff00", 0.001))], TraceCalls.FontColor("ff0000", 1.1347))], TraceCalls.FontColor("c93600", 0.7867)), TraceCalls.Trace(setindex!, (DataStructures.OrderedDict(1=>2,3=>4,1=>2,3=>4), 4, 3), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.ht_keyindex2, (DataStructures.OrderedDict(1=>2,3=>4,1=>2,3=>4), 3), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.hashindex, (3, 16), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0.0003))], TraceCalls.FontColor("00ff00", 0.0002)), TraceCalls.Trace(DataStructures._setindex!, (DataStructures.OrderedDict(1=>2,3=>4,1=>2,3=>4), 4, 3, 7), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0.0003))], TraceCalls.FontColor("00ff00", 0.0003))], TraceCalls.FontColor("ff0000", 1.4381))], TraceCalls.FontColor("ff0000", 1.0))